In [1]:
# import dependencies/libraries

from pybaseball import statcast_pitcher, playerid_lookup
import pandas as pd

In [4]:
# dictionary for each starting pitcher in the 2024 Rockies roster

pitcher_dictionary = {"Kyle Freeland": {"name": ["freeland", 'kyle'], "start_date" : "2017-01-01", "end_date":"2024-10-21"},
                       "Germán Márquez": {"name": ['marquez', 'german'],"start_date" : "2016-01-01", "end_date":"2024-10-21"},
                       "Antonio Senzatela": {"name": ['senzatela', 'antonio'], "start_date" : "2017-01-01", "end_date":"2024-10-21"},
                       "Ryan Feltner": {"name": ['feltner', 'ryan'], "start_date" : "2021-01-01", "end_date":"2024-10-21"},
                       "Chris Flexen": {"name": ['flexen', 'chris'], "start_date" : "2017-01-01", "end_date":"2024-10-21"}}

In [5]:
# define function to lookup each pitcher listed in "pitcher_database" in the pybaseball library, and return all play-by-play data for all games in each player's MLB career history.

def get_pitcher_data(pitcher_dictionary):
    all_pitcher_data = {}

    for pitcher_name, info in pitcher_dictionary.items():
        print(f"Processing data for {pitcher_name}...")
        
        # Lookup player ID
        last_name, first_name = info['name']
        pitcher_lookup = playerid_lookup(last_name, first_name)
        
        if pitcher_lookup.empty:
            print(f"Could not find player ID for {pitcher_name}. Skipping...")
            continue
        
        pitcher_id = pitcher_lookup['key_mlbam'].iloc[0]
        
        # Get play-by-play data
        start_date = info['start_date']
        end_date = info['end_date']
        
        try:
            data = statcast_pitcher(start_date, end_date, pitcher_id)
            all_pitcher_data[pitcher_name] = data
            print(f"Successfully retrieved data for {pitcher_name}")
        except Exception as e:
            print(f"Error retrieving data for {pitcher_name}: {str(e)}")
    
    return all_pitcher_data

pitcher_data = get_pitcher_data(pitcher_dictionary)

# Print summary of retrieved data
for pitcher, data in pitcher_data.items():
    print(f"{pitcher}: {len(data)} rows of data")


Processing data for Kyle Freeland...
Gathering player lookup table. This may take a moment.
Gathering Player Data
Successfully retrieved data for Kyle Freeland
Processing data for Germán Márquez...
Gathering Player Data


c:\Users\elcoo\anaconda3\Lib\site-packages\pybaseball\utils.py:295: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))


Successfully retrieved data for Germán Márquez
Processing data for Antonio Senzatela...
Gathering Player Data
Successfully retrieved data for Antonio Senzatela
Processing data for Ryan Feltner...
Gathering Player Data
Successfully retrieved data for Ryan Feltner
Processing data for Chris Flexen...
Gathering Player Data
Successfully retrieved data for Chris Flexen
Kyle Freeland: 19163 rows of data
Germán Márquez: 17257 rows of data
Antonio Senzatela: 12258 rows of data
Ryan Feltner: 5717 rows of data
Chris Flexen: 11420 rows of data


In [6]:
#convert function's output from dictionary to dataframes
# change play-by-play data keys/values into dictionary for each player
player_dict_dataframes = {player: pd.DataFrame(data) for player, data in pitcher_data.items()}

#convert each player key and values to separate dataframe
freeland_df = player_dict_dataframes['Kyle Freeland']
marquez_df = player_dict_dataframes['Germán Márquez']
senzatela_df = player_dict_dataframes['Antonio Senzatela']
feltner_df = player_dict_dataframes['Ryan Feltner']
flexen_df = player_dict_dataframes['Chris Flexen']


In [10]:
# freeland_df.to_excel("../excel/test_data.xlsx")


In [11]:
# define a function to:
## 1. sort the dataframes
## 2. create a pitch count column
## 3. create a trailing pitch column

def add_data_df(df):
    df_sort = df.sort_values(by=['game_date', 'at_bat_number','pitch_number'])
    df_sort['pitch_count'] = df_sort.groupby("game_date").cumcount()+1
    df_sort['trailing_pitch'] = df_sort.groupby("batter")['pitch_type'].shift(1)
    return df_sort

In [12]:
# sort Freeland's dataframe and add columns
freeland_df_sorted = add_data_df(freeland_df)
freeland_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
14654,FF,2017-02-28,93.5,3.20,5.68,"Freeland, Kyle",444843,607536,NaN,ball,...,0,Standard,Standard,148.0,0.000,NaN,NaN,NaN,1,NaN
14653,FF,2017-02-28,93.5,3.08,5.96,"Freeland, Kyle",444843,607536,NaN,foul,...,0,Standard,Standard,169.0,0.000,NaN,NaN,NaN,2,FF
14652,SL,2017-02-28,92.0,3.27,5.66,"Freeland, Kyle",444843,607536,NaN,foul,...,0,Standard,Standard,176.0,0.000,NaN,NaN,NaN,3,FF
14651,FF,2017-02-28,95.1,3.25,5.64,"Freeland, Kyle",444843,607536,NaN,foul,...,0,Standard,Standard,141.0,0.000,NaN,NaN,NaN,4,SL
14650,SL,2017-02-28,87.8,3.21,5.76,"Freeland, Kyle",444843,607536,strikeout,swinging_strike,...,0,Standard,Standard,193.0,0.022,NaN,NaN,NaN,5,FF


In [13]:
# sort Marquez's dataframe and add columns
marquez_df_sorted = add_data_df(marquez_df)
marquez_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
13738,FF,2016-03-13,95.2,-2.07,5.93,"Márquez, Germán",518794,608566,NaN,foul,...,0,Standard,Standard,206.0,0.000,NaN,NaN,NaN,1,NaN
13737,FF,2016-03-13,95.6,-2.14,6.10,"Márquez, Germán",518794,608566,NaN,ball,...,0,Standard,Standard,208.0,0.000,NaN,NaN,NaN,2,FF
13736,FF,2016-03-13,95.4,-2.08,5.96,"Márquez, Germán",518794,608566,NaN,ball,...,0,Standard,Standard,200.0,0.000,NaN,NaN,NaN,3,FF
13735,FF,2016-03-13,96.0,-2.06,6.07,"Márquez, Germán",518794,608566,NaN,ball,...,0,Standard,Standard,204.0,0.000,NaN,NaN,NaN,4,FF
13734,NaN,2016-03-13,NaN,NaN,NaN,"Márquez, Germán",518794,608566,field_out,hit_into_play,...,0,NaN,NaN,NaN,0.001,NaN,NaN,NaN,5,FF


In [14]:
# sort Senzatela's dataframe and add columns
senzatela_df_sorted = add_data_df(senzatela_df)
senzatela_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
11919,FF,2017-02-25,96.0,-2.76,5.91,"Senzatela, Antonio",571875,622608,NaN,ball,...,4,Standard,Standard,216.0,0.000,NaN,NaN,NaN,1,NaN
11918,FF,2017-02-25,96.1,-2.57,6.08,"Senzatela, Antonio",571875,622608,NaN,called_strike,...,4,Standard,Standard,193.0,0.000,NaN,NaN,NaN,2,FF
11917,FF,2017-02-25,95.7,-2.08,6.36,"Senzatela, Antonio",571875,622608,NaN,ball,...,4,Standard,Standard,184.0,0.000,NaN,NaN,NaN,3,FF
11916,FF,2017-02-25,96.6,-2.49,6.07,"Senzatela, Antonio",571875,622608,single,hit_into_play,...,4,Standard,Standard,198.0,0.044,NaN,NaN,NaN,4,FF
11915,FF,2017-02-25,94.4,-2.56,5.93,"Senzatela, Antonio",630111,622608,field_out,hit_into_play,...,4,Standard,Standard,195.0,-0.041,NaN,NaN,NaN,5,NaN


In [15]:
# sort Feltner's dataframe and add columns
feltner_df_sorted = add_data_df(feltner_df)
feltner_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
5716,FF,2021-09-05,91.6,-1.82,5.11,"Feltner, Ryan",645277,663372,home_run,hit_into_play,...,0,Infield shift,Strategic,223.0,-0.101,1.000,NaN,NaN,1,NaN
5715,FF,2021-09-05,93.0,-1.81,5.13,"Feltner, Ryan",624585,663372,NaN,called_strike,...,0,Infield shift,Strategic,224.0,0.000,-0.038,NaN,NaN,2,NaN
5714,SL,2021-09-05,83.9,-1.78,5.15,"Feltner, Ryan",624585,663372,NaN,ball,...,0,Infield shift,Strategic,110.0,0.000,0.027,NaN,NaN,3,FF
5713,FF,2021-09-05,91.5,-1.90,5.12,"Feltner, Ryan",624585,663372,NaN,ball,...,0,Infield shift,Strategic,216.0,0.000,0.052,NaN,NaN,4,SL
5712,FF,2021-09-05,92.6,-1.82,5.16,"Feltner, Ryan",624585,663372,NaN,foul,...,0,Infield shift,Strategic,216.0,0.000,-0.066,NaN,NaN,5,FF


In [16]:
# sort Flexen's dataframe and add columns
flexen_df_sorted = add_data_df(flexen_df)
flexen_df_sorted.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
6665,FF,2017-07-27,93.2,-1.43,6.31,"Flexen, Chris",622534,623167,NaN,called_strike,...,1,Standard,Standard,224.0,0.000,-0.037,NaN,NaN,1,NaN
6664,FF,2017-07-27,92.5,-1.55,6.33,"Flexen, Chris",622534,623167,NaN,foul,...,1,Standard,Standard,212.0,0.000,-0.056,NaN,NaN,2,FF
6663,CU,2017-07-27,76.6,-1.36,6.30,"Flexen, Chris",622534,623167,home_run,hit_into_play,...,1,Standard,Standard,8.0,0.105,1.093,NaN,NaN,3,FF
6662,FF,2017-07-27,92.5,-1.54,6.33,"Flexen, Chris",641319,623167,NaN,ball,...,1,Standard,Standard,224.0,0.000,0.036,NaN,NaN,4,NaN
6661,FF,2017-07-27,92.5,-1.60,6.33,"Flexen, Chris",641319,623167,NaN,ball,...,1,Standard,Standard,223.0,0.000,0.065,NaN,NaN,5,FF


In [19]:
# combine all individual pitcher dataframes into a single dataframe
pitchers_df = pd.concat([freeland_df_sorted, marquez_df_sorted, senzatela_df_sorted, feltner_df_sorted, flexen_df_sorted], axis=0, ignore_index=True)
pitchers_df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitch_count,trailing_pitch
0,FF,2017-02-28,93.5,3.20,5.68,"Freeland, Kyle",444843,607536,NaN,ball,...,0,Standard,Standard,148.0,0.000,NaN,NaN,NaN,1,NaN
1,FF,2017-02-28,93.5,3.08,5.96,"Freeland, Kyle",444843,607536,NaN,foul,...,0,Standard,Standard,169.0,0.000,NaN,NaN,NaN,2,FF
2,SL,2017-02-28,92.0,3.27,5.66,"Freeland, Kyle",444843,607536,NaN,foul,...,0,Standard,Standard,176.0,0.000,NaN,NaN,NaN,3,FF
3,FF,2017-02-28,95.1,3.25,5.64,"Freeland, Kyle",444843,607536,NaN,foul,...,0,Standard,Standard,141.0,0.000,NaN,NaN,NaN,4,SL
4,SL,2017-02-28,87.8,3.21,5.76,"Freeland, Kyle",444843,607536,strikeout,swinging_strike,...,0,Standard,Standard,193.0,0.022,NaN,NaN,NaN,5,FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65810,FF,2024-09-26,92.2,-0.29,6.75,"Flexen, Chris",676572,623167,single,hit_into_play,...,7,Standard,Standard,191.0,-0.005,0.365,63.06783,6.09774,92,FF
65811,FF,2024-09-26,91.0,-0.12,6.92,"Flexen, Chris",592273,623167,NaN,foul,...,7,Standard,Standard,191.0,0.000,-0.063,73.46046,6.96244,93,FF
65812,CU,2024-09-26,70.7,-0.04,6.97,"Flexen, Chris",592273,623167,NaN,called_strike,...,7,Standard,Standard,20.0,0.000,-0.073,NaN,NaN,94,FF
65813,SL,2024-09-26,79.4,-0.16,6.83,"Flexen, Chris",592273,623167,strikeout,swinging_strike,...,7,Standard,Standard,24.0,0.005,-0.225,25.47238,6.91687,95,CU


In [20]:
# export master pitcher dataframe to csv
pitchers_df.to_csv("../data/all_pitcher_data.csv")

In [24]:
# look at list of columns in the master dataframe
column_list = pitchers_df.columns.to_list()
column_list

['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'description',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
 'game_type',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'fielder_2',
 'umpire',
 'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimat

In [25]:
# confirm length of list of columns
len(column_list)

96